In [ ]:
!pip install swifter
import spacy
import re
import pandas as pd
import nltk
nltk.download(['punkt_tab','stopwords','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from itertools import chain
from collections import Counter
from spacy import displacy,tokenizer
import swifter

en_stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df = pd.read_csv('bbc_news.csv')
df.head()


,Unnamed: 0,index,title,pubDate,guid,link,description
0,0,6684,Can I refuse to work?,"Wed, 10 Aug 2022 15:46:18 GMT",https://www.bbc.co.uk/news/business-62147992,https://www.bbc.co.uk/news/business-62147992?a...,With much of the UK enduring another period of...
1,1,9267,'Liz Truss the Brief?' World reacts to UK poli...,"Mon, 17 Oct 2022 11:35:12 GMT",https://www.bbc.co.uk/news/world-63285480,https://www.bbc.co.uk/news/world-63285480?at_m...,The UK's political chaos has been watched arou...
2,2,7387,Rationing energy is nothing new for off-grid c...,"Wed, 31 Aug 2022 05:20:18 GMT",https://www.bbc.co.uk/news/uk-scotland-highlan...,https://www.bbc.co.uk/news/uk-scotland-highlan...,Scoraig in the north west Highlands has long h...
3,3,767,The hunt for superyachts of sanctioned Russian...,"Tue, 22 Mar 2022 14:37:01 GMT",https://www.bbc.co.uk/news/60739336,https://www.bbc.co.uk/news/60739336?at_medium=...,"Wealthy Russians sanctioned by the US, EU and ..."
4,4,3712,Platinum Jubilee: 70 years of the Queen in 70 ...,"Wed, 01 Jun 2022 23:17:33 GMT",https://www.bbc.co.uk/news/uk-61660128,https://www.bbc.co.uk/news/uk-61660128?at_medi...,A quick look back at the Queen's 70 years on t...


In [ ]:
titles = pd.DataFrame(df['title'])
titles.head()

,title
0,Can I refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...
2,Rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...


In [ ]:
titles['no_punc_title'] = titles['title'].apply(lambda x : re.sub(r'[^\w\s]','',x))
titles.head()

,title,no_punc_title
0,Can I refuse to work?,Can I refuse to work
1,'Liz Truss the Brief?' World reacts to UK poli...,Liz Truss the Brief World reacts to UK politic...
2,Rationing energy is nothing new for off-grid c...,Rationing energy is nothing new for offgrid co...
3,The hunt for superyachts of sanctioned Russian...,The hunt for superyachts of sanctioned Russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...,Platinum Jubilee 70 years of the Queen in 70 s...


In [ ]:
titles['lower_punc_title'] = titles['title'].apply(lambda x : re.sub(r'[^\w\s]','',x)).str.lower()
titles.head()

,title,no_punc_title,lower_punc_title
0,Can I refuse to work?,Can I refuse to work,can i refuse to work
1,'Liz Truss the Brief?' World reacts to UK poli...,Liz Truss the Brief World reacts to UK politic...,liz truss the brief world reacts to uk politic...
2,Rationing energy is nothing new for off-grid c...,Rationing energy is nothing new for offgrid co...,rationing energy is nothing new for offgrid co...
3,The hunt for superyachts of sanctioned Russian...,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...,Platinum Jubilee 70 years of the Queen in 70 s...,platinum jubilee 70 years of the queen in 70 s...


In [ ]:
titles['no_stopwords'] = titles['lower_punc_title'].apply(lambda x : ' '.join([word for word in x.split() if word not in en_stopwords]))
titles.head()

,title,no_punc_title,lower_punc_title,no_stopwords
0,Can I refuse to work?,Can I refuse to work,can i refuse to work,refuse work
1,'Liz Truss the Brief?' World reacts to UK poli...,Liz Truss the Brief World reacts to UK politic...,liz truss the brief world reacts to uk politic...,liz truss brief world reacts uk political turmoil
2,Rationing energy is nothing new for off-grid c...,Rationing energy is nothing new for offgrid co...,rationing energy is nothing new for offgrid co...,rationing energy nothing new offgrid community
3,The hunt for superyachts of sanctioned Russian...,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs
4,Platinum Jubilee: 70 years of the Queen in 70 ...,Platinum Jubilee 70 years of the Queen in 70 s...,platinum jubilee 70 years of the queen in 70 s...,platinum jubilee 70 years queen 70 seconds


In [ ]:
titles['tokenized_title'] = titles['no_stopwords'].apply(lambda x : word_tokenize(x))
titles.head()

,title,no_punc_title,lower_punc_title,no_stopwords,tokenized_title
0,Can I refuse to work?,Can I refuse to work,can i refuse to work,refuse work,"[refuse, work]"
1,'Liz Truss the Brief?' World reacts to UK poli...,Liz Truss the Brief World reacts to UK politic...,liz truss the brief world reacts to uk politic...,liz truss brief world reacts uk political turmoil,"[liz, truss, brief, world, reacts, uk, politic..."
2,Rationing energy is nothing new for off-grid c...,Rationing energy is nothing new for offgrid co...,rationing energy is nothing new for offgrid co...,rationing energy nothing new offgrid community,"[rationing, energy, nothing, new, offgrid, com..."
3,The hunt for superyachts of sanctioned Russian...,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,"[hunt, superyachts, sanctioned, russian, oliga..."
4,Platinum Jubilee: 70 years of the Queen in 70 ...,Platinum Jubilee 70 years of the Queen in 70 s...,platinum jubilee 70 years of the queen in 70 s...,platinum jubilee 70 years queen 70 seconds,"[platinum, jubilee, 70, years, queen, 70, seco..."


In [ ]:
lm = WordNetLemmatizer()
ps = PorterStemmer()

titles['lemmatized_title'] = titles['tokenized_title'].swifter.apply(lambda x : [lm.lemmatize(word) for word in x])
titles['stemmed_title'] = titles['tokenized_title'].swifter.apply(lambda x : [ps.stem(word) for word in x])
titles.head()

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

,title,no_punc_title,lower_punc_title,no_stopwords,tokenized_title,lemmatized_title,stemmed_title
0,Can I refuse to work?,Can I refuse to work,can i refuse to work,refuse work,"[refuse, work]","[refuse, work]","[refus, work]"
1,'Liz Truss the Brief?' World reacts to UK poli...,Liz Truss the Brief World reacts to UK politic...,liz truss the brief world reacts to uk politic...,liz truss brief world reacts uk political turmoil,"[liz, truss, brief, world, reacts, uk, politic...","[liz, truss, brief, world, reacts, uk, politic...","[liz, truss, brief, world, react, uk, polit, t..."
2,Rationing energy is nothing new for off-grid c...,Rationing energy is nothing new for offgrid co...,rationing energy is nothing new for offgrid co...,rationing energy nothing new offgrid community,"[rationing, energy, nothing, new, offgrid, com...","[rationing, energy, nothing, new, offgrid, com...","[ration, energi, noth, new, offgrid, commun]"
3,The hunt for superyachts of sanctioned Russian...,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,"[hunt, superyachts, sanctioned, russian, oliga...","[hunt, superyachts, sanctioned, russian, oliga...","[hunt, superyacht, sanction, russian, oligarch]"
4,Platinum Jubilee: 70 years of the Queen in 70 ...,Platinum Jubilee 70 years of the Queen in 70 s...,platinum jubilee 70 years of the queen in 70 s...,platinum jubilee 70 years queen 70 seconds,"[platinum, jubilee, 70, years, queen, 70, seco...","[platinum, jubilee, 70, year, queen, 70, second]","[platinum, jubile, 70, year, queen, 70, second]"


In [ ]:
cleaned_list = list(chain.from_iterable(titles['lemmatized_title']))
cleaned_list[:10]

['refuse',
 'work',
 'liz',
 'truss',
 'brief',
 'world',
 'reacts',
 'uk',
 'political',
 'turmoil']

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(' '.join(cleaned_list))

pos_df = pd.DataFrame(columns=['token','pos'])
for word in doc:
  pos_df = pd.concat([pos_df,pd.DataFrame.from_records([{'token' : word.text, 'pos': word.pos_}])], ignore_index = True)
pos_df.head()

,token,pos
0,refuse,AUX
1,work,NOUN
2,liz,PROPN
3,truss,ADJ
4,brief,ADJ


In [ ]:
pos_df_count = pos_df.groupby(['token','pos']).size().reset_index(name='counts').sort_values(by='counts', ascending = False)
pos_df_count.head()

,token,pos,counts
30,2022,NUM,47
1151,england,PROPN,45
860,cup,PROPN,39
3032,say,VERB,38
3676,uk,PROPN,37


In [ ]:
Verb = pos_df_count[pos_df_count['pos'] == 'VERB']
Verb.head()

,token,pos,counts
3032,say,VERB,38
3680,ukraine,VERB,22
354,beat,VERB,13
2118,make,VERB,13
1368,found,VERB,13


In [ ]:
for word in doc.ents:
  print(word.text,word.label_)

russian NORP
70 year DATE
70 second TIME
bull ORG
1 CARDINAL
georgia taylorbrown womens ORG
terry hall PERSON
uk GPE
six CARDINAL
99 CARDINAL
jubilee beacon PERSON
5 CARDINAL
russian NORP
week DATE
1117 june 2022 DATE
hong kong GPE
25 year DATE
nick pope PERSON
newcastle GPE
china GPE
madrid GPE
clasico GPE
kuenssberg GPE
100yearold CARDINAL
three day week DATE
600kg QUANTITY
uk home ukrainian refugee nicola bulley ORG
democrat NORP
republican NORP
european NORP
dutch 17yearold PERSON
three CARDINAL
missouri GPE
four CARDINAL
scottish NORP
chris PERSON
jr v liam smith PERSON
russian NORP
ton england ORG
west indie NORP
166 CARDINAL
last eight DATE
2022 CARDINAL
bellingham gakpo fernandez breakout star PERSON
briton jack fenton killed PERSON
greek NORP
england v india edgbaston PERSON
rwanda GPE
lenny henry PERSON
lib dems PERSON
chris kaba PERSON
2022 DATE
james webb PERSON
nasa ORG
constitution hill cheltenham ORG
33 CARDINAL
2022 CARDINAL
vittorio pozzos PERSON
six month DATE
100 CAR

In [ ]:
displacy.render(doc,style = 'ent' , jupyter = True)

In [ ]:
unprocessed_text = ' '.join(titles['no_punc_title'])

doc1 = nlp(unprocessed_text)

token_list = [word.lemma_ for word in doc1 if not word.is_stop and word.is_alpha]
ner_list = [[word.text , word.label_] for word in doc1.ents]

print(token_list[:10])
print(ner_list[:10])

['refuse', 'work', 'Liz', 'Truss', 'Brief', 'World', 'react', 'UK', 'political', 'turmoil']
[['Liz Truss', 'PERSON'], ['UK', 'GPE'], ['Rationing', 'PRODUCT'], ['superyachts', 'CARDINAL'], ['Russian', 'NORP'], ['Platinum', 'ORG'], ['70 years', 'DATE'], ['70 seconds', 'TIME'], ['Red Bull', 'ORG'], ['Formula', 'ORG']]


In [ ]:
# Sample column: df['titles'] = ["Can Abdul refuse to work?"]
def process_with_ner_pos(text):
    doc = nlp(text.lower())

    result = []
    for token in doc:
        if token.is_alpha and not token.is_stop:  # Skip punctuation, numbers, etc.
            lemma = token.lemma_
            # Check if token is part of a Named Entity
            ner_type = token.ent_type_ if token.ent_type_ else 'O'  # 'O' = Outside any entity
            result.append([lemma, token.pos_, ner_type])
    return result

# Apply to your DataFrame
df['processed'] = df['title'].apply(process_with_ner_pos)
df['processed'].head(10)

,processed
0,"[[refuse, VERB, O], [work, VERB, O]]"
1,"[[liz, PROPN, O], [trus, VERB, O], [brief, NOU..."
2,"[[ration, VERB, O], [energy, NOUN, O], [new, A..."
3,"[[hunt, NOUN, O], [superyacht, NOUN, CARDINAL]..."
4,"[[platinum, NOUN, PERSON], [jubilee, PROPN, PE..."
5,"[[red, PROPN, O], [bull, NOUN, ORG], [find, VE..."
6,"[[world, PROPN, O], [triathlon, PROPN, O], [ch..."
7,"[[terry, PROPN, PERSON], [hall, PROPN, PERSON]..."
8,"[[post, VERB, O], [office, NOUN, O], [fujitsu,..."
9,"[[pavement, NOUN, O], [parking, NOUN, O], [fri..."
